In [9]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import csv

In [10]:
df = pd.read_csv("../Sorted_combined.csv",header=None,dtype={0: 'str',
 1: 'float64',
 2: 'float64',
 3: 'float64',
 4: 'float64',
 5: 'float64',
 6: 'float64',
 7: 'float64',
 8: 'float64',
 9: 'float64',
 10: 'float64',
 11: 'float64',
 12: 'float64',
 13: 'float64',
 14: 'float64',
 15: 'float64',
 16: 'float64',
 17: 'float64',
 18: 'float64',
 19: 'int64',
 20: 'str'})

In [11]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1554535986.0,-1.647343,-1.956522,-1.562802,-1.115942,-1.084541,-1.118357,-0.971014,127.000000,-0.979311,...,0.081189,-0.146538,-0.239349,0.136632,-0.960292,-1.289528,-0.144415,0.17516,0,Kuntal
1,1554535986.0,-1.647343,-1.956522,-1.562802,-1.115942,-1.084541,-1.118357,-0.971014,127.000000,-0.979311,...,0.081189,-0.146538,-0.239349,0.136632,-0.960292,-1.289528,-0.144415,0.17516,0,Shailaja
2,1554535989.0,-2.745819,-3.035117,-3.526756,-1.448161,-1.683946,-0.836120,-0.968227,-32.822742,-0.980840,...,0.080267,-0.141640,-0.224180,0.139840,-0.964293,-1.098293,0.004593,0.57038,0,Kuntal
3,1554535989.0,-2.745819,-3.035117,-3.526756,-1.448161,-1.683946,-0.836120,-0.968227,-32.822742,-0.980840,...,0.080267,-0.141640,-0.224180,0.139840,-0.964293,-1.098293,0.004593,0.57038,0,Shailaja
4,1554535992.0,-1.618333,-1.761667,-1.893333,-1.003333,-0.115000,-0.823333,-0.950000,-122.951667,-0.979620,...,0.075060,-0.145587,-0.242633,0.124593,-0.961480,-1.113340,-0.805847,0.17794,0,Kuntal


In [12]:
dtype_map = {}
for index,dt in enumerate(list(df.dtypes)):
    dtype_map[index]=dt.name
dtype_map.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [13]:
df[19].value_counts()

0    74489
1     3181
Name: 19, dtype: int64

In [14]:
df[20].value_counts()

Sandipan    27614
Shailaja    21101
Pratyay     16835
Kuntal      12120
Name: 20, dtype: int64

In [15]:
df.groupby(20)[19].value_counts()

20        19
Kuntal    0     11558
          1       562
Pratyay   0     15375
          1      1460
Sandipan  0     26761
          1       853
Shailaja  0     20795
          1       306
Name: 19, dtype: int64

In [20]:
# lets Create Sequence of data from this
# algo, group by user , sort by id take k rows at a time, 
from collections import deque
import sys

users = list(df[20].unique())
window_size = 60
window_lens = [5,10,15,30,60]
new_data = []
with open(str(window_size)+'_all_output.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    for user in tqdm(users[0:1]):
        part_df = df[df[20]==user].values.tolist()
        q = deque()
        idq = deque()
        for index,row in tqdm(enumerate(part_df[0:61]),total=10):
            if len(q) < window_size:
                q.append(row)
                idq.append(str(index))
            else:
                copied_list = []
                for row in q:
                    copied_list.append(row)
                features = []
                for windows in window_lens:
                    temp_list = copied_list[-windows:]
                    part_df = pd.DataFrame.from_records(temp_list,columns=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
                    f_l=np.array(part_df.describe().values.tolist()[1:]).reshape(7*19).tolist()
                    features = features+f_l
                    
                label = q[-1][19]
                data = [user+"-"+''.join(idq)]+features+[label]
                print(data)
                csvwriter.writerow(data)
                q.popleft()
                q.append(row)
                idq.popleft()
                idq.append(str(index))

100%|██████████| 1/1 [00:00<00:00,  3.78it/s]

['Kuntal-01234567891011121314151617181920212223242526272829303132333435363738394041424344454647484950515253545556575859', -1.3320453111637822, -1.7601186399189026, -1.5846768751310623, -0.9679778536027441, -0.918255130722741, -1.1482547962543586, -0.8319968813957725, -9.549645090119407, -0.9649742639821032, 0.10258248769574949, 0.10172725727069361, -0.21915745861297542, -0.23462597762863538, 0.1645837046979866, -0.9568507919463087, -1.2298381655480983, -0.12186263087248324, 0.3268383176733781, 0.0, 0.5282654863342431, 0.7081688778372305, 0.6741929036479705, 0.22133142420137997, 0.4545908183973721, 0.3268932780379534, 0.16434828127725254, 110.53848290049122, 0.0010087768805879444, 0.006105526615892437, 0.0005712919052385598, 0.0019064613726987479, 0.012461873070666827, 0.004388426366991827, 0.0020076754226653858, 0.10731130588396473, 0.2664129431858633, 0.32076827056634144, 0.0, -2.1187290969899664, -2.68561872909699, -2.6638795986622075, -1.294314381270903, -1.5719063545150502, -1.5866

In [ ]:
# lets Create Sequence of data from this
# algo, group by user , sort by id take k rows at a time, 

